<a href="https://colab.research.google.com/github/madarasw/Advanced_deep_learning_module/blob/main/ViTs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test 1


In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [61]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"madarawimalarathna","key":"998571907b3220ce6ac5e9cd10df1deb"}'}

In [63]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [64]:
!ls -l ~/.kaggle

total 4
-rw------- 1 root root 74 Nov  6 09:03 kaggle.json


In [71]:
!kaggle datasets download -d ciplab/real-and-fake-face-detection

Dataset URL: https://www.kaggle.com/datasets/ciplab/real-and-fake-face-detection
License(s): CC-BY-NC-SA-4.0
 96% 412M/431M [00:00<00:00, 412MB/s]
100% 431M/431M [00:00<00:00, 488MB/s]


In [82]:
!unzip real-and-fake-face-detection.zip -d data/

Archive:  real-and-fake-face-detection.zip
  inflating: data/real_and_fake_face/training_fake/easy_100_1111.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_101_0010.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_102_0101.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_103_1111.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_104_1000.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_105_1100.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_106_0011.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_107_1011.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_108_1111.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_109_1111.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_10_0001.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_110_0001.jpg  
  inflating: data/real_and_fake_face/training_fake/easy_111_1101.jpg  
  inflating: data/real_and_fake_fac

In [111]:
!ls /content/data/real_and_fake_face

training_fake  training_real


In [74]:
!pip install transformers timm datasets accelerate torchvision


In [113]:
# ============================================================
# ✅ IMPORTS
# ============================================================

import os
import shutil
from pathlib import Path
from typing import Dict, Tuple

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import (
    ViTForImageClassification,
    AutoImageProcessor,
    get_linear_schedule_with_warmup
)

from tqdm import tqdm


In [114]:
# ============================================================
# ✅ CONFIG — EDIT IF NEEDED
# ============================================================

RAW_DATA_ROOT = "/content/data/real_and_fake_face_detection/real_and_fake_face"
PROCESSED_DATA = "/content/data_split"   # ✅ where ImageFolder structure will go

MODEL_NAME = "google/vit-base-patch16-224"
OUTPUT_DIR = "/content/vit_output"

IMAGE_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 4
LR = 2e-5
WEIGHT_DECAY = 0.01
VAL_SPLIT = 0.2
WARMUP_STEPS = 100
NUM_WORKERS = 2
SEED = 42


In [116]:
# ============================================================
# ✅ Dataset Preparation (runs once)
# ============================================================

def prepare_dataset():
    raw_real = list(Path(f"{RAW_DATA_ROOT}/training_real").glob("*"))
    raw_fake = list(Path(f"{RAW_DATA_ROOT}/training_fake").glob("*"))

    print(f"Found {len(raw_real)} REAL images")
    print(f"Found {len(raw_fake)} FAKE images")

    train_real, val_real = train_test_split(raw_real, test_size=VAL_SPLIT, random_state=SEED)
    train_fake, val_fake = train_test_split(raw_fake, test_size=VAL_SPLIT, random_state=SEED)

    # Create folders
    for split in ["train", "val"]:
        for cls in ["real", "fake"]:
            os.makedirs(f"{PROCESSED_DATA}/{split}/{cls}", exist_ok=True)

    # Copy files
    for p in train_real:
        shutil.copy(p, f"{PROCESSED_DATA}/train/real/")
    for p in train_fake:
        shutil.copy(p, f"{PROCESSED_DATA}/train/fake/")
    for p in val_real:
        shutil.copy(p, f"{PROCESSED_DATA}/val/real/")
    for p in val_fake:
        shutil.copy(p, f"{PROCESSED_DATA}/val/fake/")

    print("✅ Dataset prepared successfully!")

# Run only once
prepare_dataset()


Found 1081 REAL images
Found 960 FAKE images
✅ Dataset prepared successfully!


In [117]:
# ============================================================
# ✅ Build Transform
# ============================================================

def build_transforms(image_size):
    train_tf = T.Compose([
        T.Resize((image_size, image_size)),
        T.RandomHorizontalFlip(),
        T.RandomRotation(10),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])
    ])

    val_tf = T.Compose([
        T.Resize((image_size, image_size)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])
    ])

    return train_tf, val_tf


def create_loaders():
    train_tf, val_tf = build_transforms(IMAGE_SIZE)

    train_dataset = ImageFolder(f"{PROCESSED_DATA}/train", transform=train_tf)
    val_dataset = ImageFolder(f"{PROCESSED_DATA}/val", transform=val_tf)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=NUM_WORKERS, pin_memory=True)

    return train_loader, val_loader


In [118]:
# ============================================================
# ✅ Evaluation
# ============================================================

def evaluate(model, dataloader, device):
    model.eval()
    preds, trues = [], []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images).logits
            pred = outputs.argmax(dim=1)

            preds.extend(pred.cpu().tolist())
            trues.extend(labels.cpu().tolist())

    acc = accuracy_score(trues, preds)
    prec, rec, f1, _ = precision_recall_fscore_support(
        trues, preds, average="binary", zero_division=0
    )

    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}


In [122]:
# ============================================================
# ✅ TRAINING LOOP
# ============================================================

def train_vit():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader, val_loader = create_loaders()

    print(f"Loading model: {MODEL_NAME}")

    model = ViTForImageClassification.from_pretrained(
      MODEL_NAME,
      num_labels=2,
      ignore_mismatched_sizes=True   # ✅ important fix
    ).to(device)


    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    total_steps = len(train_loader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(
        optimizer, WARMUP_STEPS, total_steps
    )

    best_f1 = 0
    scaler = torch.cuda.amp.GradScaler()

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    for epoch in range(1, EPOCHS + 1):
        model.train()
        pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")

        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                loss = model(images, labels=labels).loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            pbar.set_postfix(loss=float(loss))

        # --------- Eval ---------
        metrics = evaluate(model, val_loader, device)
        print(f"Eval: {metrics}")

        # Save each epoch
        model.save_pretrained(f"{OUTPUT_DIR}/epoch{epoch}")

        # Save best model
        if metrics["f1"] > best_f1:
            best_f1 = metrics["f1"]
            model.save_pretrained(f"{OUTPUT_DIR}/best_model")

            print(f"✅ New best model (F1 = {best_f1:.4f})")

    print("Training finished.")


In [123]:

train_vit()

Loading model: google/vit-base-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1106988423.py:26: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/4:   0%|          | 0/102 [00:00<?, ?it/s]/tmp/ipython-input-1106988423.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/tmp/ipython-input-110

Eval: {'accuracy': 0.58679706601467, 'precision': 0.7105263157894737, 'recall': 0.37327188940092165, 'f1': 0.48942598187311176}
✅ New best model (F1 = 0.4894)


Epoch 2/4:   0%|          | 0/102 [00:00<?, ?it/s]/tmp/ipython-input-1106988423.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 2/4: 100%|██████████| 102/102 [00:21<00:00,  4.82it/s, loss=0.395]


Eval: {'accuracy': 0.7114914425427873, 'precision': 0.6911196911196911, 'recall': 0.8248847926267281, 'f1': 0.7521008403361344}
✅ New best model (F1 = 0.7521)


Epoch 3/4:   0%|          | 0/102 [00:00<?, ?it/s]/tmp/ipython-input-1106988423.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 3/4: 100%|██████████| 102/102 [00:22<00:00,  4.60it/s, loss=0.643]


Eval: {'accuracy': 0.7334963325183375, 'precision': 0.7327586206896551, 'recall': 0.783410138248848, 'f1': 0.7572383073496659}
✅ New best model (F1 = 0.7572)


Epoch 4/4:   0%|          | 0/102 [00:00<?, ?it/s]/tmp/ipython-input-1106988423.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 4/4: 100%|██████████| 102/102 [00:22<00:00,  4.59it/s, loss=0.469]


Eval: {'accuracy': 0.7579462102689487, 'precision': 0.7836538461538461, 'recall': 0.7511520737327189, 'f1': 0.7670588235294118}
✅ New best model (F1 = 0.7671)
Training finished.


In [121]:
!rm -rf /content/vit_output